In [260]:
import numpy as np
import pandas as pd
from scipy.stats import kurtosis, skew, multivariate_normal
from scipy.signal import welch, find_peaks
import matplotlib.pyplot as plt
from scipy.ndimage import uniform_filter1d
from scipy.fft import fftfreq, fft
from scipy import stats
%matplotlib tk

In [12]:
V_SF = pd.read_csv('MotorVRB_2025.csv')
V_2F = pd.read_csv('MotorV2F_2025.csv')

<center><h1>Time-Domain Features</center></h1>

In [13]:
def calculate_rms(signal):
    return np.sqrt(np.mean(signal**2))

def extract_signal_stats(signal, window_size=1680):
    start = 0
    stats_list = []
    epsilon = 1e-8  # Small value to avoid division by zero

    while start < len(signal):
        end = min(start + window_size, len(signal))
        cycle = signal[start:end]

        mean_val = np.mean(cycle)
        rms_val = calculate_rms(cycle)
        peak_val = np.max(np.abs(cycle))

        stats_list.append({
            'Kurtosis': kurtosis(cycle, fisher=True, bias=False),
            'Skewness': skew(cycle, bias=False),
            'Mean': mean_val,
            'Peak': peak_val,
            'Variance': np.var(cycle),
            'StandardDeviation': np.std(cycle),
            'RMS': rms_val,
            'ShapeFactor': rms_val / (np.abs(mean_val) + epsilon),
            'ImpulseFactor': peak_val / (np.abs(mean_val) + epsilon),
        })

        start += window_size  # Move to next window

    df_stats = pd.DataFrame(stats_list)
    return df_stats

In [132]:
RMS_0 = extract_signal_stats(V_SF['Sinal-C0'])
RMS_20 = extract_signal_stats(V_SF['Sinal-C20'])
RMS_40 = extract_signal_stats(V_SF['Sinal-C40'])
RMS_60 = extract_signal_stats(V_SF['Sinal-C60'])
RMS_80 = extract_signal_stats(V_SF['Sinal-C80'])
RMS_100 = extract_signal_stats(V_SF['Sinal-C100'])

In [292]:
RMS_F0 = extract_signal_stats(V_2F['Sinal-C0'])
RMS_F20 = extract_signal_stats(V_2F['Sinal-C20'])
RMS_F40 = extract_signal_stats(V_2F['Sinal-C40'])
RMS_F60 = extract_signal_stats(V_2F['Sinal-C60'])
RMS_F80 = extract_signal_stats(V_2F['Sinal-C80'])
RMS_F100 = extract_signal_stats(V_2F['Sinal-C100'])

<center><h1>Frequency-Domain Features</center></h1>

In [19]:
def extract_frequency_features(signal, Fs=50000, window_size=1680):
    """
    Extracts frequency-domain features from a signal.

    Features:
    - Spectral Centroid
    - Spectral Bandwidth
    Parameters:
    - signal: 1D numpy array of the signal
    - Fs: Sampling frequency in Hz
    - window_size: Number of samples per analysis window

    Returns:
    - Pandas DataFrame with features for each window
    """
    epsilon = 1e-8
    start = 0
    features = []

    while start < len(signal):
        end = min(start + window_size, len(signal))
        segment = signal[start:end]
        # Compute Power Spectral Density (PSD)
        freqs, psd = welch(segment, fs=Fs, nperseg=min(256, len(segment)))
        # Spectral Centroid
        spectral_centroid = np.sum(freqs * psd) / (np.sum(psd) + epsilon)
        # Spectral Bandwidth
        spectral_bandwidth = np.sqrt(np.sum(((freqs - spectral_centroid)**2) * psd) / (np.sum(psd) + epsilon))

        features.append({
            'SpectralCentroid': spectral_centroid,
            'SpectralBandwidth': spectral_bandwidth,
        })

        start += window_size  # Next window

    return pd.DataFrame(features)

In [293]:
RMS_SF = np.concatenate([RMS_0['RMS'], RMS_20['RMS'], RMS_40['RMS'],
                         RMS_60['RMS'], RMS_80['RMS'], RMS_100['RMS']])

In [294]:
RMS_2F = np.concatenate([RMS_F0['RMS'], RMS_F20['RMS'], RMS_F40['RMS'],
                         RMS_F60['RMS'], RMS_F80['RMS'], RMS_F100['RMS']])

In [296]:
plt.figure()
plt.plot(RMS_SF, linewidth=0.5, label='Healthy')
plt.plot(RMS_2F, linewidth=0.5, label='Faulty')
plt.legend(loc='best')
plt.show()

In [262]:
time_domain = extract_signal_stats(V_SF['Sinal-C100'])

In [289]:
time_domain2 = extract_signal_stats(V_2F['Sinal-C100'])

<h2>Feature Statistics at 80% Load DataFrame

In [34]:
time_domain.to_csv('TD_VSF80.csv', index=False)
freq_domain.to_csv('FD_VSF80.csv', index=False)

<h2> Step 2: Expand PF State Vector for Feature Tracking

In [337]:
def train_particle_model(RMS_SF, N=10000):
    ''' Initialization '''
    state_dim = 2
    ''' Initialize particles and weights '''
    particles = np.zeros((N, state_dim))
    weights = np.ones(N) / N

    # Inicialización en base a señal saludable
    rms_min, rms_max = RMS_SF.min(), RMS_SF.max()
    particles[:, 0] = np.random.uniform(rms_min, rms_max, N)
    particles[:, 1] = np.random.uniform(0, 1, N)

    # Estimar dinámica
    estimated_rms = np.zeros(len(RMS_SF))
#    estimated_health = np.zeros(len(true_rms))


    rms_process_noise = 0.0005
    health_process_noise = 0.002
    measurement_noise = 0.001
    alpha = 0.99
    for t in range(len(RMS_SF)):
        z_t = RMS_SF[t]

        # Predicción
        mean_prev = np.mean(particles[:, 0])
        particles[:, 0] = alpha * particles[:, 0] + (1 - alpha) * mean_prev + np.random.normal(0, rms_process_noise, size=N)
        particles[:, 1] += np.random.normal(0, health_process_noise, size=N)
        particles[:, 1] = np.clip(particles[:, 1], 0, 1)

        # Actualización
        particle_rms = particles[:, 0]
        weights = np.exp(-0.5 * ((particle_rms - z_t) / measurement_noise) ** 2)
        weights += 1e-300
        weights /= np.sum(weights)

        # Re-muestreo
        indices = np.random.choice(N, size=N, p=weights)
        particles = particles[indices]
        weights.fill(1.0 / N)

        # Estimación
        x_t = np.mean(particles, axis=0)
        estimated_rms[t] = x_t[0]

    # Cálculo del error saludable (natural)
    error_sano = np.abs(RMS_SF - estimated_rms)
    tolerancia = np.mean(error_sano) + 0.8 * np.std(error_sano)
    return particles, tolerancia, rms_process_noise, health_process_noise


In [338]:
def predict_rms_without_observation(particles_init, length, rms_process_noise, health_process_noise):
    N, state_dim = particles_init.shape
    particles = particles_init.copy()
    estimated_rms = np.zeros(length)

    for t in range(length):
        # Solo predicción, sin observar nada
        particles[:, 0] += np.random.normal(0, rms_process_noise, size=N)
        particles[:, 1] += np.random.normal(0, health_process_noise, size=N)
        particles[:, 1] = np.clip(particles[:, 1], 0, 1)

        indices = np.random.choice(N, size=N)
        particles = particles[indices]

        x_t = np.mean(particles, axis=0)
        estimated_rms[t] = x_t[0]

    return estimated_rms


In [342]:
# Entrenamiento con condiciones saludables
particles_model, tolerancia, pn_rms, pn_health = train_particle_model(RMS_SF)

# Señal a evaluar (con o sin falla)
true_rms = RMS_SF

# Predicción sin observar la señal
estimated_rms = predict_rms_without_observation(particles_model,
                                                len(true_rms),
                                                pn_rms,
                                                pn_health)

# Detección de falla por desviación de la forma
error = np.abs(estimated_rms - true_rms)
falla_detectada = error > tolerancia


In [343]:
# entrenás con RMS_SF (saludable)
particles_model, _, pn_rms, pn_health = train_particle_model(RMS_SF)

# y calculás tolerancia local solo en esa fase
def calcular_tolerancia_local_entrenamiento(error_entrenamiento, length, window_size=500, k=3):
    tolerancia_local = np.zeros(length)
    for i in range(length):
        inicio = max(0, i - window_size // 2)
        fin = min(len(error_entrenamiento), i + window_size // 2)
        ventana = error_entrenamiento[inicio:fin]
        media = np.mean(ventana)
        std = np.std(ventana)
        tolerancia_local[i] = media + k * std
    return tolerancia_local

# obtené el error saludable durante entrenamiento
error_sano = np.abs(RMS_SF - predict_rms_without_observation(particles_model, len(RMS_SF), pn_rms, pn_health))

# extendés la tolerancia a toda la evaluación (rellenás si es necesario)
tolerancia_local = calcular_tolerancia_local_entrenamiento(error_sano, len(RMS_2F))

# luego aplicás el detector
estimated_rms = predict_rms_without_observation(particles_model, len(RMS_2F), pn_rms, pn_health)
error = np.abs(estimated_rms - RMS_2F)
falla_detectada = error > tolerancia_local

In [344]:
plt.figure(figsize=(14, 6))
plt.plot(true_rms, label='RMS real (evaluada)', linewidth=0.8)
plt.plot(estimated_rms, label='RMS esperada (modelo saludable)', linestyle='--')

# Banda superior e inferior de tolerancia
plt.plot(estimated_rms + tolerancia_local, color='orange', linestyle=':', linewidth=1, label='Tolerancia superior')
plt.plot(estimated_rms - tolerancia_local, color='orange', linestyle=':', linewidth=1)

plt.scatter(np.where(falla_detectada)[0],
            true_rms[falla_detectada],
            color='red', s=15, label='Falla detectada')

plt.xlabel('Time Step')
plt.ylabel('RMS')
plt.legend()
plt.tight_layout()
plt.show()


In [351]:
import numpy as np
import matplotlib.pyplot as plt


def train_particle_model(RMS_SF, N=10000, alpha=0.99, rms_process_noise=0.005, health_process_noise=0.002,
                         measurement_noise=0.001):
    state_dim = 2
    particles = np.zeros((N, state_dim))
    weights = np.ones(N) / N

    # Inicialización con rango de la señal
    rms_min, rms_max = RMS_SF.min(), RMS_SF.max()
    particles[:, 0] = np.random.uniform(rms_min, rms_max, N)
    particles[:, 1] = np.random.uniform(0, 1, N)

    estimated_rms = np.zeros(len(RMS_SF))

    for t in range(len(RMS_SF)):
        z_t = RMS_SF[t]
        mean_prev = np.mean(particles[:, 0])
        particles[:, 0] = alpha * particles[:, 0] + (1 - alpha) * mean_prev + np.random.normal(0, rms_process_noise,
                                                                                               size=N)
        particles[:, 1] += np.random.normal(0, health_process_noise, size=N)
        particles[:, 1] = np.clip(particles[:, 1], 0, 1)

        # Actualización
        particle_rms = particles[:, 0]
        weights = np.exp(-0.5 * ((particle_rms - z_t) / measurement_noise) ** 2)
        weights += 1e-300
        weights /= np.sum(weights)

        indices = np.random.choice(N, size=N, p=weights)
        particles = particles[indices]
        weights.fill(1.0 / N)

        x_t = np.mean(particles, axis=0)
        estimated_rms[t] = x_t[0]

    error_sano = np.abs(RMS_SF - estimated_rms)
    tolerancia = np.mean(error_sano) + 3 * np.std(error_sano)
    return particles, tolerancia, rms_process_noise, health_process_noise


def predict_rms_without_observation(particles_init, length, rms_process_noise, health_process_noise):
    N, _ = particles_init.shape
    particles = particles_init.copy()
    estimated_rms = np.zeros(length)

    for t in range(length):
        particles[:, 0] += np.random.normal(0, rms_process_noise, size=N)
        particles[:, 1] += np.random.normal(0, health_process_noise, size=N)
        particles[:, 1] = np.clip(particles[:, 1], 0, 1)

        indices = np.random.choice(N, size=N)
        particles = particles[indices]
        estimated_rms[t] = np.mean(particles[:, 0])

    return estimated_rms


def particle_filter_window(RMS_total, window_size=1548, step_size=1548, N=10000,
                           alpha=0.99, k_tol=3, rms_noise=0.005, health_noise=0.002, meas_noise=0.001):
    total_len = len(RMS_total)
    estimated_global = np.zeros(total_len)
    tolerancia_global = np.zeros(total_len)
    falla_global = np.zeros(total_len, dtype=bool)
    conteo_uso = np.zeros(total_len)

    for start in range(0, total_len - window_size + 1, step_size):
        end = start + window_size
        ventana = RMS_total[start:end]

        # Entrenar modelo en la ventana
        particles, _, pn_rms, pn_health = train_particle_model(
            ventana, N=N, alpha=alpha,
            rms_process_noise=rms_noise,
            health_process_noise=health_noise,
            measurement_noise=meas_noise
        )

        # Predicción sin observar la ventana
        estimated_rms = predict_rms_without_observation(particles, window_size, pn_rms, pn_health)

        # Error y tolerancia local
        error = np.abs(estimated_rms - ventana)
        tolerancia_local = np.mean(error) + k_tol * np.std(error)
        falla_local = error > tolerancia_local

        # Acumulación
        estimated_global[start:end] += estimated_rms
        tolerancia_global[start:end] += tolerancia_local
        falla_global[start:end] |= falla_local
        conteo_uso[start:end] += 1

    # Promedio en zonas superpuestas
    estimated_global /= np.maximum(conteo_uso, 1)
    tolerancia_global /= np.maximum(conteo_uso, 1)

    return estimated_global, tolerancia_global, falla_global


# Señal completa (puede tener fallas)
RMS_total = RMS_SF  # o RMS_SF si es saludable

# Ejecutar análisis por ventanas de 1548
estimated_rms, tolerancia_local, falla_detectada = particle_filter_window(
    RMS_total,
    window_size=1548,
    step_size=1548,  # sin solapamiento
    N=5000,  # podés ajustar este número
    alpha=0.99,
    k_tol=3
)

plt.figure(figsize=(14, 6))
plt.plot(RMS_total, label='RMS real', linewidth=0.8)
plt.plot(estimated_rms, label='RMS estimada (modelo saludable)', linestyle='--')
plt.plot(estimated_rms + tolerancia_local, color='orange', linestyle=':', linewidth=1, label='Tolerancia superior')
plt.plot(estimated_rms - tolerancia_local, color='orange', linestyle=':', linewidth=1)

plt.scatter(np.where(falla_detectada)[0],
            RMS_total[falla_detectada],
            color='red', s=15, label='Falla detectada')

plt.xlabel('Time Step')
plt.ylabel('RMS')
plt.legend()
plt.tight_layout()
plt.show()


In [346]:
RMS_SF.shape

(9288,)